開一個 github:
- Simple RAG With Sreamlit: https://python.langchain.com/docs/tutorials/rag/, https://python.langchain.com/docs/how_to/streaming/
- Conversational RAG: https://python.langchain.com/docs/how_to/message_history/, https://python.langchain.com/docs/tutorials/qa_chat_history/ 
- LCEL 介紹: https://python.langchain.com/docs/how_to/#langchain-expression-language-lcel
- Tool: https://python.langchain.com/docs/how_to/#tools


# Pip

In [5]:
%pip install --quiet --upgrade langchain langchain-community langchain-chroma langchain-openai
%pip isntall python-dotenv
%pip install bs4


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: unknown command "isntall" - maybe you meant "install"
Note: you may need to restart the kernel to use updated packages.
  Using cached bs4-0.0.2-py2.py3-none-any.whl.metadata (411 bytes)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
Using cached bs4-0.0.2-py2.py3-none-any.whl (1.2 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Import packages

In [1]:
import os
from dotenv import load_dotenv

load_dotenv("/Users/wenye/ML&AI Tutor/.env.local")

OPENAI_KEY = os.getenv("OPENAI_KEY")

# Simple RAG

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_KEY)

In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()
docs

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistake

In [8]:
print(f"第一筆資料是：")
docs[0]

第一筆資料是：


Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes

In [13]:
# Split the documents into chunks and index them.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# 試著調整 chunk_size 和 chunk_overlap 的數值，看看結果有何不同


splits = text_splitter.split_documents(docs)
print(
    f"Split finished. There are {len(splits)} chunks.\n---\nThe first chunk is:\n{splits[0]}"
)

Split finished. There are 66 chunks.
---
The first chunk is:
page_content='LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby impr

創建一個 Chroma 向量資料庫物件，並使用 OpenAIEmbeddings 作為嵌入器


In [ ]:
import numpy as np

# 先建立 openai embedding(我們這次用 openai 的，但都可以換)
OPENAI_EMBEDDING = OpenAIEmbeddings(api_key=OPENAI_KEY)
result = OPENAI_EMBEDDING.embed_documents(texts=[splits[0].page_content])

print(f"第一個 chunk: \n{splits[0].page_content}\n-----\n")
print(f"第一個 chunk 的嵌入向量形狀是是：{np.array(result).shape}\n向量是：\n{result}")

第一個 chunk: 
LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.


Memory
-----

第一個 chunk 的

In [26]:
# 用試著將前兩個 chunks 的內容嵌入向量化
result = OPENAI_EMBEDDING.embed_documents(
    texts=[splits.page_content for splits in splits[:2]]
)

print(f"chunks: \n{[splits.page_content for splits in splits[:2]]}\n-----\n")
print(f"chunks 的嵌入向量形狀是是：{np.array(result).shape}\n向量是：\n{result}")

chunks: 
['LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.\n\n\nMemory', 'M

In [ ]:
# 建立 Chroma 向量資料庫
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents=splits, embedding=OPENAI_EMBEDDING)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",  # default: similarity
    search_kwargs={
        "k": 3,
        "score_threshold": 0.5,
        "filter": {"source": "https://lilianweng.github.io/posts/2023-06-23-agent/"},
    },
)  # 這裡可以調整參數，可以點擊進去原始套件程式碼看看有哪些參數可以調整！

# 可以使用 invoke 方法來搜尋相關的文章
retriever.invoke("請問 React 搭建 agent 的方式是什麼？")

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='}\n]\nThen after these clarification, the agent moved into the code writing mode with a different system message.\nSystem message:'),
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='}\n]\nThen after these clarification, the agent moved into the code writing mode with a different system message.\nSystem message:'),
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='}\n]\nThen after these clarification, the agent moved into the code writing mode with a different system message.\nSystem message:')]

In [37]:
# 我們抓取 prompt
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

/Users/wenye/ML&AI Tutor/.venv/lib/python3.11/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:", additional_kwargs={}, response_metadata={})]

In [38]:
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [56]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# 這裡是我們的 langchain（LCEL語法）的流程
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = rag_chain.invoke("What is Task Decomposition?")
print(f"response: \n{response}")

response: 
Task Decomposition is a technique where complex tasks are broken down into smaller, simpler steps. This process is often facilitated by the Chain of Thought (CoT) prompting method, which encourages models to "think step by step." It enhances model performance and provides insight into the model’s reasoning process.


In [40]:
# 如果是用 stream 方法的話，可以逐步看到結果（執行看看！）
for chunk in rag_chain.stream("What is Task Decomposition?"):
    print(chunk, end="", flush=True)

Task Decomposition is the process of breaking down complex tasks into smaller, more manageable steps. It often utilizes the Chain of Thought (CoT) prompting technique, where a model is instructed to "think step by step" to enhance performance. This approach not only simplifies complex tasks but also provides insight into the model's reasoning process.

In [ ]:
# 這裡是我們的 langchain（LCEL語法）的流程
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    # | llm
    # | StrOutputParser()
)

response = rag_chain.invoke("What is Task Decomposition?")
print(f"response: \n{response}")

# 觀察結果：已經有 retrive 出相關的文章在 prompt 裏面了

response: 
messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: What is Task Decomposition? \nContext: Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\n\nFig. 1. Overview of a LLM-powered autonomous agent syst

In [ ]:
# 這裡是我們的 langchain（LCEL語法）的流程
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    # | StrOutputParser()
)

response = rag_chain.invoke("What is Task Decomposition?")
print(f"response: \n{response}")

# 觀察結果：回答是完整的，還需要進行 parser

response: 
content='Task Decomposition is a process where complex tasks are broken down into smaller, manageable steps. This is often achieved using the Chain of Thought (CoT) technique, which prompts models to "think step by step," enhancing their ability to tackle difficult tasks. It not only simplifies the tasks but also provides insight into the model\'s reasoning process.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 436, 'total_tokens': 504, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None} id='run-e60eac79-7e6f-4c97-a940-a641d9b519ca-0' usage_metadata={'input_tokens': 436, 'output_tokens': 68, 'total_tokens': 504, 'input_token_details': {'

如果要回傳參考的文本呢？有兩個方法

In [ ]:
# 1. 使用非LCEL語法的方式來建立 chain，使用 langchain_core 的套件

from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


# 自己設定 prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# 建立 chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)
result = rag_chain.invoke({"input": "What is Task Decomposition?"})
result

{'input': 'What is Task Decomposition?',
 'context': [Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.'),
  Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many ste

In [68]:
# 2. 使用 LCEL 語法的方式來建立 chain

# 這個方法上面都是一樣的，不同的是，我們多創建了一個 retriever 的 Runnable，這個是用來搜尋相關的文章
# 也就是說我們是分開 retriever 和 問答 的 Runnable，這樣可以存取 retriever 的結果
# 順序：retriever -> 將結果放入問答

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# This Runnable takes a dict with keys 'input' and 'context',
# formats them into a prompt, and generates a response.
rag_chain_from_docs = (
    {
        "input": lambda x: x["input"],  # input query
        "context": lambda x: format_docs(x["context"]),  # context
    }
    | prompt  # format query and context into prompt
    | llm  # generate response
    | StrOutputParser()  # coerce to string
)

# Pass input query to retriever
retrieve_docs = (lambda x: x["input"]) | retriever
print(f"retrieve_docs: \n{retrieve_docs}\n---\n")

# Below, we chain `.assign` calls. This takes a dict and successively
# adds keys-- "context" and "answer"-- where the value for each key
# is determined by a Runnable. The Runnable operates on all existing
# keys in the dict.

# 因為我們是將先有問答的結果 retrieve_docs，放入問答的 chain 裏面，要用 assign 的方式來放入
chain = RunnablePassthrough.assign(context=retrieve_docs)
print(f"chain: \n{chain}\n---\n")

chain = chain.assign(
    answessss=rag_chain_from_docs  # answer key 可以隨便取, 因為是新建的
)

print(f"chain: \n{chain}\n---\n")

# 觀察這個 chain, 很像是我們要設定 dict 包含 等等要傳入的 input, 以及 context 和 answer，
# 然後將 retrieve_docs 放入 context，將 rag_chain_from_docs 放入 answer
# 一開始只有 input, context 兩個 key,
# 而因為 answer 是新建的 key 用來放入問答的結果，所以我們可以看到 assign 的方式是將新建的 key 放入

chain.invoke({"input": "What is Task Decomposition"})

retrieve_docs: 
first=RunnableLambda(lambda x: x['input']) middle=[] last=VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x10f029590>, search_type='similarity_score_threshold', search_kwargs={'k': 3, 'score_threshold': 0.5, 'filter': {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}})
---

chain: 
mapper={
  context: RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x10f029590>, search_type='similarity_score_threshold', search_kwargs={'k': 3, 'score_threshold': 0.5, 'filter': {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}})
}
---

chain: 
first=RunnableAssign(mapper={
  context: RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x10f029590>, searc

{'input': 'What is Task Decomposition',
 'context': [Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.'),
  Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many step

# Conversational RAG

# Use Streamlit as Interface

https://github.com/wsxqaza12/RAG_LangChain_streamlit/blob/main/rag_engine.py